In [1]:
import numpy as np
from matplotlib import pylab as plt

# Function to calculate the image size

Setting the dimensions of the image:
- plt.figure() method with the argument 'figsize': plt.figure(figsize = (8,6))
- dimensions are a tupla of numbers: (width, hight) in inches

Relationship between the array dimension and the image dimension:
- the image size depends on the size of the array (number of rows, number of columns)
- array pixels / cells are square and dimensionless
- the size depends on how many pixels (columns, rows) per unit of length (1 inch) are - this parameter is `ppi` (ixels per inch)
- for the resolution for printing, an equivalent dpi parameter is used `dpi` (dots per inch)
- the larger the dpi, the smaller the image and vice versa:

$$ \begin{align}
width &=& number\_of\_columns * 1/ppi\\
hight &=& number\_of\_rows * 1/ppi
\end{align}$$

In [2]:
def mySize(arrList,dpi = 100):
    ''' Only up to 6 images, 3 images per row.
        Args:
            - arrList:   list of numpy arrays
            - dpi:       piksel for inch
    '''
    dpi = 1/dpi
    h = max([x.shape[0] for x in arrList])
    w = max([x.shape[1] for x in arrList])
    l = len(arrList)
    if l == 1:
        height = h
        width = w
    elif l <= 3:
        height = h
        width = w * l
    elif l > 3 and l < 7:
        height = h * 2
        width = w * 3
    return int(width * dpi), int(height * dpi)

# OTSU’S METHOD FOR IMAGE THESHOLDING

The digital image has pixels with different values associated with different objects. The distribution of pixel values is characterized by the mean value $\mu$ and variance $\sigma^2$.


The method is based on the assumption that there are two separating groups of pixel values (classes) in the image: for background and for xxx. The algorithm looks for a place on the histogram in which the division can occur.
Each class has its own distribution of pixel values. The place of division is a place for which:
- the sum of variances within groups ($\sigma_w^2$) is minimal

or
- variance between groups ($\sigma_p$) is maximum

In [3]:
def myOtsu(ar):
    ar = ar.copy()
    ar = ar.ravel() # converts 2D arrays to 1 dimensional data vector
    mu = ar.mean()    # total mean value
    N = ar.size
    K = 2
    pixelVal = np.arange(256)
    threshold =[0,0] # collects two values: [sigma, threshold]
    
    # data to the graph as variance changes
    tt = [] # collects threshold values
    sg =[]  # collects variances
   
    for t in pixelVal: 
        cl1 = ar[(ar < t)]
        cl2 = ar[ar >= t]
        
        #if any class has zero elements - skip this threshold
        if cl1.size ==0 or cl2.size == 0:
            continue
        
        N1,N2 = cl1.size, cl2.size
        w1,w2 = N1/N, N2/N  # calculate weight
        mu1,mu2 = cl1.mean(), cl2.mean()
        
        sigma_p = (w1*(mu1-mu)**2 + w2*(mu2-mu)**2)/(K-1)
        sigma_p = np.round(sigma_p,0)
              
        tt.append(t)
        sg.append(int(sigma_p))
        
        # it starts with the value threshold[0] = 0
        if sigma_p > threshold[0]:
            threshold[0]= sigma_p
            threshold[1] = t  
    del ar

    return (tt,sg,threshold)

# Simple plotting function

During the exercises, images are displayed, usually from 2 to 6. In order not to write an image loop every time, the code used will be saved as a function:
  - displays up to 6 images
  - 1 or 2 rows of three images each

In [4]:
def myplot(images,titles,size=None,hist=None):
    ''' images - list of images (np.arrays)
        titles - list of image titles
        size   - tuple of image size in inch eg. (10,12)
    '''
    images = images[:]
    if len(images) > 3:
        rows,cols = 2, 6
    else:
        rows, cols= 1, len(images)
    if size:
        f = plt.figure(figsize=size)
    else:
        f = plt.figure()
    for i,img in enumerate(images,1): # i - image number from 1
        plt.subplot(rows,cols,i)
        if hist:
            plt.hist(img.ravel(),bins=256)
        else:
            plt.imshow(img, cmap=plt.cm.Greys_r)
            plt.axis('off')
        plt.title(titles[i-1])
        
    f.tight_layout()

# Stretching
Simple function for histogram stretching:
  - line function
  - manual selection of stretch limits

In [5]:
def myStretch(ar,t):
    ''' Args:
            - ar:   numpy 2D array like image
            - t:    tuple (a, b), a: threshold on the left
                                  b: threshold on the right'''
    ar = ar.copy()
    
    # transformation coefficients: point 1: t, point 2: (245,255)
    a,b = np.polyfit(t,[0,255],1)
    print(f'Transformation coefficients:\n{"a:":>15} {a}, b: {b}\n')
    x = np.unique(ar.ravel())
    y = a*x + b

    new_ar = ar * a + b
    new_ar[ar<0] = 0
    new_ar[ar>255] = 255
    new_ar = np.array(new_ar,dtype=np.uint8)
    
    return new_ar

# Conversion to grayscale

In [ ]:
def rgbToGray(ar,weights=[0.11,0.59,0.3]):
    gr = ar.copy()
    weights = weights[:]
    
    for i,ww in enumerate(weights):
        gr[:,:,i] = gr[:,:,i] * ww
    
    gr = np.array(np.sum(gr,axis=2),dtype=np.uint8)
    return gr

# Histogram equalization
Simple function for Histogram equalization:
 - calculate CDF (cumulative distributive function)
 - multiply the CDF value with gray levels (for 8 bits = 256-1 = 255)

In [6]:
def myEqual(ar,c=0):
    ''' Args:
            - ar:   numpy 2D array like image'''
    ar = ar.copy()
